In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight') # Optional: for a nicer plot style

Step 3: Get Stock Data

In [ ]:
# Define stock ticker and date range
ticker = "RELIANCE.NS"  # Reliance Industries NSE
start_date = "2020-01-01" # Adjust as needed for historical data
end_date = "2023-12-31"   # Adjust as needed

# Download historical data using yfinance
data = yf.download(ticker, start=start_date, end=end_date)

# Display the first few rows of the data
print(data.head())

Step 4: Calculate Returns and Volatility

In [ ]:
# Calculate daily percentage returns
data['Daily_Return'] = data['Adj Close'].pct_change()
data = data.dropna() # Remove the first row with NaN (due to pct_change)

# Calculate volatility (standard deviation of daily returns)
volatility = data['Daily_Return'].std() * np.sqrt(252) # Annualize volatility (assuming 252 trading days in a year)

# Calculate drift (average daily return)
drift = data['Daily_Return'].mean() * 252 # Annualize drift

print(f"Annual Volatility: {volatility:.4f}")
print(f"Annual Drift: {drift:.4f}")

Step 5: Monte Carlo Simulation Function

In [ ]:
def monte_carlo_simulation(last_price, days, num_simulations, drift, volatility):
    """
    Performs a Monte Carlo simulation for stock price prediction using GBM.

    Args:
        last_price (float): The last known stock price.
        days (int): Number of days to predict.
        num_simulations (int): Number of simulation paths to generate.
        drift (float): Annualized drift (average return).
        volatility (float): Annualized volatility.

    Returns:
        numpy.ndarray: A 2D array of simulated price paths.
                       Shape: (num_simulations, days + 1) - includes initial price.
    """
    dt = 1/252  # Time step (daily)
    price_paths = np.zeros((num_simulations, days + 1))
    price_paths[:, 0] = last_price # Set initial price for all simulations

    for i in range(num_simulations):
        for j in range(1, days + 1):
            # Generate random shock (Z ~ N(0, 1))
            Z = np.random.normal(0, 1)
            # GBM formula
            price_paths[i, j] = price_paths[i, j-1] * np.exp((drift - 0.5 * volatility**2) * dt + volatility * np.sqrt(dt) * Z)

    return price_paths

Step 6: Run the Simulation

In [ ]:
# Simulation parameters
last_known_price = data['Adj Close'].iloc[-1] # Last available adjusted close price
prediction_days = 30 # Number of days to predict
num_simulations = 1000 # Number of simulation paths

# Run the Monte Carlo simulation
simulated_paths = monte_carlo_simulation(last_known_price, prediction_days, num_simulations, drift, volatility)

In [ ]:
Step 7: Visualize Results

In [ ]:
# Plotting the simulated paths
plt.figure(figsize=(12, 6))
for i in range(num_simulations):
    plt.plot(simulated_paths[i, :], linewidth=0.5, alpha=0.3) # Plot each path with transparency

plt.title(f'Monte Carlo Simulation for {ticker} - {num_simulations} Paths')
plt.xlabel('Days')
plt.ylabel('Stock Price')
plt.grid(True)
plt.show()

# Histogram of ending prices (price on the last predicted day)
ending_prices = simulated_paths[:, -1]
plt.figure(figsize=(8, 5))
plt.hist(ending_prices, bins=50, density=True, alpha=0.6, color='skyblue')
plt.title(f'Distribution of Predicted Prices after {prediction_days} Days')
plt.xlabel('Stock Price')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

Step 8: Analyze Results

In [ ]:
# Calculate average predicted price
average_predicted_price = np.mean(ending_prices)
print(f"Average Predicted Price after {prediction_days} days: {average_predicted_price:.2f}")

# Calculate confidence interval (e.g., 95% confidence interval)
confidence_interval = np.percentile(ending_prices, [2.5, 97.5]) # 2.5th and 97.5th percentiles for 95% CI
print(f"95% Confidence Interval for Predicted Price: {confidence_interval}")